In [1]:
import csv
import json
import numpy as np
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder  # Add this import statement
import random
import pickle

In [2]:
class fileSystem:
    @staticmethod
    def readCSV(filename):
        id_list = []
        sepal_length_list = []
        sepal_width_list = []
        petal_length_list = []
        petal_width_list = []
        species_list = []
        with open(filename + '.csv', 'r') as csv_file:
            csv_reader = csv.reader(csv_file)
            next(csv_reader)
            for row in csv_reader:
                id = row[0]
                sepal_length = row[1].split(',')  # Split layers string into list
                sepal_width = row[2].split(',')  
                petal_length = row[3].split(',')
                petal_width = row[4].split(',')
                species = row[5].split(',')
                
                id_list.append(id)
                sepal_length_list.append(sepal_length)
                sepal_width_list.append(sepal_width)
                petal_length_list.append(petal_length)
                petal_width_list.append(petal_width)
                species_list.append(species)

        return id_list, sepal_length_list, sepal_width_list, petal_length_list, petal_width_list, species_list

    def readJSON(filename):
        with open("testcaseB/" + filename + ".json", 'r') as file:
            data = json.load(file)
            input_size = data["case"]["model"]["input_size"]
            layer = data["case"]["model"]["layers"]
            weights = data["case"]["initial_weights"]
            input_array = np.array(data["case"]["input"])
            target = np.array(data["case"]["target"])
            learning_rate = data["case"]["learning_parameters"]["learning_rate"]
            batch_size = data["case"]["learning_parameters"]["batch_size"]
            max_iteration = data["case"]["learning_parameters"]["max_iteration"]
            error_threshold = data["case"]["learning_parameters"]["error_threshold"]
            num_layer = len(layer)

        return input_size, layer, weights, input_array, target, learning_rate, batch_size, max_iteration, error_threshold, num_layer

In [16]:
import numpy as np

class forwardActivation:
    def relu(x):
        return np.maximum(0, x)

    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    def linear(x):
        return x

    def softmax(x):
        exp_values = np.exp(x - np.max(x, axis=-1, keepdims=True))
        return exp_values / np.sum(exp_values, axis=-1, keepdims=True)

class backActivationOutput:
    def relu(x):
        return 1 * (x >= 0)

    def linear(x):
        return np.ones_like(x)

    def sigmoid(x):
        return x * (1 - x)
    
    def softmax(x):
        return x * (1 - x)

class backActivationHidden:
    def relu(x, y):
        return np.where(y < 0, 0, -(x - y))
    
    def linear(x, y):
        return -(x - y)

    def sigmoid(x, y):
        return -y * (1 - y) * (x - y)
    
    def softmax(x, y):
        return x * (1 - x)


In [85]:
class Layer:
    def __init__(self, neurons, activation, weights=None, bias=1):
        self.neurons = neurons
        self.activation_name = activation
        self.activation_func = forwardActivation.__dict__[activation]
        self.activation_derivative_output = backActivationOutput.__dict__[activation]
        self.activation_derivative_hidden = backActivationHidden.__dict__[activation]
        self.weights = weights
        self.bias = bias
        self.outputs = None  # Initialize outputs attribute to None

    def set_outputs(self, outputs):
        self.outputs = outputs

class NeuralNetwork:
    def __init__(self, learning_rate=0.1, batch_size=10, max_iterations=100, error_threshold=0.1):
        self.layers = []
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.max_iteration = max_iterations
        self.error_threshold = error_threshold

    def add_layer(self, num_neurons, activation_function, weights, bias):
        self.layers.append(Layer(
            neurons=num_neurons,
            activation=activation_function,
            weights=weights,
            bias=bias,
        ))

    def forward_pass(self, input_array):
        outputs = input_array
        for i, layer in enumerate(self.layers):
            layer_output = layer.activation_func(np.dot(outputs, layer.weights) + layer.bias)
            layer.set_outputs(layer_output)  # Set outputs attribute
            outputs = layer_output
        return outputs

    def calculate_loss(self, outputs, targets):
        return np.mean(np.square(targets - outputs))

    def backward_pass(self, input_array, targets, outputs):
        delta = -(targets - outputs) * self.layers[-1].activation_derivative_output(outputs)
        for i in range(len(self.layers) - 1, -1, -1):
            layer = self.layers[i]

            if i != 0:
                delta_weights = np.dot(self.layers[i - 1].outputs.T, delta)
            else:
                delta_weights = np.dot(input_array.T, delta)
            layer.weights -= self.learning_rate * delta_weights
            
            delta_bias = np.sum(delta, axis=0)
            layer.bias -= self.learning_rate * delta_bias    

            if i > 0:
                delta = np.dot(delta, self.layers[i].weights.T) * self.layers[i].activation_derivative_hidden(self.layers[i - 1].outputs, outputs)

    def train(self, input_array, targets):
        for epoch in range(self.max_iteration):
            for i in range(0, len(input_array), self.batch_size):
                input_batch = input_array[i:i + self.batch_size]
                target_batch = targets[i:i + self.batch_size]

                # Forward pass for the entire batch
                outputs = self.forward_pass(input_batch)

                # Backward pass for the entire batch
                self.backward_pass(input_batch, target_batch, outputs)

                # Calculate loss for the entire batch
                loss = self.calculate_loss(outputs, target_batch)

                print(f"Iteration {epoch + 1}, Loss: {loss}")

                # Check for early stopping
                if loss < self.error_threshold:
                    print("Training converged: Error threshold reached.")
                    print("Final Weights:")
                    for i, layer in enumerate(self.layers):
                        print(f"Layer {i+1}:")
                        print(layer.bias)
                        print(layer.weights)
                    return
                    
            if epoch == self.max_iteration - 1:
                print("Training stopped by max iteration.")
        
        print("Final Weights:")
        for i, layer in enumerate(self.layers):
            print(f"Layer {i+1}:")
            print(layer.bias)
            print(layer.weights)

    def addLayer(self, layer: Layer, input_size=None) -> None:
        if not self.layers:
            if input_size is None:
                raise ValueError("First layer must define input_size explicitly.")
            prev_neurons = input_size
        else:
            prev_neurons = self.layers[-1].neurons  
        if layer.weights is None:
            layer.weights = np.random.randn(prev_neurons, layer.neurons) * 0.01
        if layer.bias is None:
            layer.bias = np.zeros(layer.neurons)
        self.layers.append(layer)


    def save_model(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump(self, f)
    
    def load_model(filename):
        with open(filename, 'rb') as f:
            return pickle.load(f)
    
    def fit(self, input_array, targets):
        for _ in range(self.max_iteration):
            for i in range(0, len(input_array), self.batch_size):
                input_batch = input_array[i:i + self.batch_size]
                target_batch = targets[i:i + self.batch_size]

                # Forward pass for the entire batch
                outputs = self.forward_pass(input_batch)

                # Backward pass for the entire batch
                self.backward_pass(input_batch, target_batch, outputs)

                # Calculate loss for the entire batch
                loss = np.mean(np.square(target_batch - outputs))

                if loss < self.error_threshold:
                    return "Training converged: Error threshold reached."

        return "Training stopped by max iteration."
    
    def predict(self, input_array):
        outputs = self.forward_pass(input_array)
        return outputs


file_name = 'linear'
input_size, layers, weights, input_array, targets, learning_rate, batch_size, max_iteration, error_threshold, num_layer = fileSystem.readJSON(file_name)
nn = NeuralNetwork(learning_rate=learning_rate, batch_size=batch_size, max_iterations=max_iteration, error_threshold=error_threshold)

for layer, weight in zip(layers, weights):
    nn.add_layer(layer["number_of_neurons"], activation_function=layer["activation_function"], weights=np.array(weight[1:]), bias=np.array(weight[0]))

nn.train(input_array, targets)
nn.save_model('model.pickle')
loaded_nn = NeuralNetwork.load_model('model.pickle')
print("\nLoaded Neural Network:")
for i, layer in enumerate(loaded_nn.layers):
    print(f"Layer {i+1}:")
    print(f"Neurons: {layer.neurons}")
    print(f"Activation Function: {layer.activation_name}")
    print(f"Weights:")
    print(layer.weights)
    print(f"Bias: {layer.bias}")

Iteration 1, Loss: 0.22166666666666668
Training stopped by max iteration.
Final Weights:
Layer 1:
[0.22 0.36 0.11]
[[ 0.64  0.3  -0.89]
 [ 0.28 -0.7   0.37]]

Loaded Neural Network:
Layer 1:
Neurons: 3
Activation Function: linear
Weights:
[[ 0.64  0.3  -0.89]
 [ 0.28 -0.7   0.37]]
Bias: [0.22 0.36 0.11]


# Perbandingan dengan SKLearn

In [107]:
iris = load_iris()
X_iris = iris.data
y_iris = iris.target

X_train, X_val, y_train, y_val = train_test_split(X_iris, y_iris, test_size=0.3, random_state=42)

model_iris_sklearn = MLPClassifier(hidden_layer_sizes=(4,), activation="logistic", learning_rate_init=0.1,
                                   batch_size=10, max_iter=1000, solver="sgd", random_state=42)
model_iris_sklearn.fit(X_train, y_train)
predictions_sklearn = model_iris_sklearn.predict(X_val)

report_sklearn = classification_report(y_val, predictions_sklearn)
print(report_sklearn)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      0.54      0.70        13
           2       0.68      1.00      0.81        13

    accuracy                           0.87        45
   macro avg       0.89      0.85      0.84        45
weighted avg       0.91      0.87      0.86        45



In [101]:
mlp_custom = NeuralNetwork(learning_rate=0.1, max_iterations=1000, batch_size=10, error_threshold=0.01)
mlp_custom.addLayer(Layer(neurons=4, activation='sigmoid'), input_size=X_train.shape[1])  # Hidden layer
mlp_custom.addLayer(Layer(neurons=y_train_onehot.shape[1], activation='softmax'))  # Output layer
y_train_onehot = np.eye(3)[y_train]  # Convert target to one-hot encoding
stop_reason = mlp_custom.fit(X_train, y_train_onehot)

predictions_custom = np.argmax(mlp_custom.predict(X_val), axis=1)

report_custom = classification_report(y_val, predictions_custom)
print(report_custom)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      0.85      0.92        13
           2       0.87      1.00      0.93        13

    accuracy                           0.96        45
   macro avg       0.96      0.95      0.95        45
weighted avg       0.96      0.96      0.96        45



<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=620db8da-30e5-4498-b1a7-3d24bc2b369d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>